# Playing against the intelligent agent

This notebook allow someone to test the graphic environnement and try de play against the trained agent.

IMPORTANT: To run this notebook you need csv file containing the Q-table. This can be done by running the python file main.py. It is initially configured to train the agent through 100.000 before saving the table, which will few minutes.

In [1]:
import pygame
from random import randint
from mastermind import Mastermind
from AI_agent import Sarsa
from main import open_table
import time
import sys
import socket
import config
import numpy as np
import itertools
import random
from collections import Counter


pygame.init()

print("Loading q_table... Please wait")
# Reading table previously learned
q_table_dict = open_table(file_name="learnt_table_SARSA_passed.csv")

# Creating an instance of an agent. Already trained (using table learned before)
RL = Sarsa(q_table=q_table_dict, is_qtable=True)

print("Q_table loaded, agent initialized")

# Creating environment instance
env = Mastermind(human=True)
pygame.display.set_caption('GET SET GO PLAYER2!')
def parse_combi(combi_string,act_table):
    
    action_num = int(combi_string[:len(combi_string)-2])
    
        
    return act_table[action_num]
def score(p, q):
        '''feedback given during the mastermind game'''
        hits = sum(p_i == q_i for p_i, q_i in zip(p, q))
        misses = sum((Counter(p) & Counter(q)).values()) - hits
        return hits, misses


def play():
    player2_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    player2_socket.connect(("localhost", 12347))
    message = player2_socket.recv(1024).decode()
    print(message)
    ready = env.ready
    flag=0
    placed=0
    try:
        while True:
            
            
            for ev in pygame.event.get():
                if ev.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif ev.type == pygame.MOUSEBUTTONDOWN:
                    if env.palet_rect.collidepoint(ev.pos) and len(env.combi) < 4:
                        x, y = ev.pos
                        x -= env.palet_rect.x
                        y -= env.palet_rect.y
                        try:
                            if env.palet_mask.get_at((x, y)):
                                env.combi.append(x // 30)
                                env.screen.blit(env.palette, (80 + (len(env.combi) - 1) * 30, 20),
                                                (x // 30 * 30, 0, 30, 30))
                                pygame.display.flip()
                        except:
                            continue
                    elif env.reset_rect.collidepoint(ev.pos):
                        env.combi = []
                        pygame.display.set_caption('GET SET GO!')
                        env.screen.blit(env.mmbg, (0, 0))
                        env.screen.blit(env.palette, env.palet_rect)
                        pygame.display.flip()
                        flag=0
                        
                    elif env.go_rect.collidepoint(ev.pos):
                        if len(env.combi) != 4:
                            env.combi = [randint(0, 5), randint(0, 5), randint(0, 5), randint(0, 5)]
                            for i in range(4):
                                env.screen.blit(env.palette, (80 + (i) * 30, 20), (env.combi[i] * 30, 0, 30, 30))
                                pygame.display.flip()
                        if flag==0:         
                            l=[str(i+1) for i in env.combi]
                            player2_socket.send(''.join(l).encode())
                            secret1 = player2_socket.recv(1024).decode()
                            print(f"Player 1's secret number: {int(secret1)}")
                            flag=1
                                           
                                # time.sleep(1)
    
                        ready = True
                        # Deploy AI agent function here
    
                        observation = "init"
                        for ligne in range(6):
                            
                            message=player2_socket.recv(1024).decode()
                            if message[0]=='g':
                                guess1=message[1:]
                                print(guess1)
                                a,b = score(secret1, guess1)
                                player2_socket.send(f"{a}{b}".encode())
                            message=player2_socket.recv(1024).decode()
                            if message == "Guessed1":
                                pygame.display.set_caption('LOST!')
                                break
                                
                            action = RL.choose_action(observation)
                            ans = env.step(action, ligne)
                            prediction = parse_combi(ans[0], env.list_action)
                            print(prediction)
                            feedback = env.feedback(prediction)
                            observation = str(action) + str(feedback[0]) + str(feedback[1])
                            
                            for i in range(4):
                                r = env.screen.blit(env.palette, ((i + 1) * 30 + 20, ligne * 35 + 70),
                                                    (prediction[i] * 30, 0, 30, 30))
                                pygame.display.update(r)
                            g="g"+str(prediction)
                            player2_socket.send(g.encode())
                            g="g"+str(env.combi)
                            player2_socket.send(g.encode())
                            fb = player2_socket.recv(1024).decode()
                            print(f"{''.join(l)}, {prediction}, {fb[0]}, {fb[1]}")
                            placed, misplaced = int(fb[0]),int(fb[1])
                            for e, c in enumerate([(255, 0, 0)] * placed + [(255, 255, 255)] * misplaced):
                                r = pygame.draw.circle(env.screen, c, (190 + e * 10, ligne * 35 + 85), 2, 0)
                                pygame.display.update(r)
                            time.sleep(1)
    
                            if placed == 4:
                                try:
                                    player2_socket.send("Guessed2".encode())
                                    pygame.display.set_caption('WON!')
                                except socket.error as e:
                                    print(f"Error sending message: {e}")
                                
                                break
                            try:
                                player2_socket.send("Try2".encode())
                            except socket.error as e:
                                print(f"Error sending message: {e}")
                                
                        if message=="Guessed1" or placed==4:
                            break
                    else:
                        break
    except KeyboardInterrupt:
        print("Interrupted")
    
        

play()
pygame.quit()
sys.exit()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading q_table... Please wait
Q_table loaded, agent initialized
Starting1
Player 1's secret number: 4231
5311
[0, 0, 2, 0]
4354, [0, 0, 2, 0], 0, 1
10168
[1, 1, 1, 2]
4354, [1, 1, 1, 2], 0, 1
2358
[2, 4, 4, 3]
4354, [2, 4, 4, 3], 2, 1
5142
[3, 2, 4, 3]
4354, [3, 2, 4, 3], 4, 0


SystemExit: 

C:\Users\Aishika Nandi\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
